In [18]:
from Gaussian import Gaussian
from EM import EM
from NN import NN
from Visual import Visual

import os
import numpy as np

if not os.path.exists('special'): os.mkdir('special')

D = 2     # dimension of sample data point
K = 3     # number of Gaussian / classifications

# parameters for generating sample
# prior probability, mu, and covariance
N_set = [np.random.randint(20000, 30000) for k in range(K-1)]
N_set.insert(0, int(sum(N_set)) )    # background
N_set = np.array(N_set)
mu_set = np.array([[0.0, 0.0],
                 [-1.0, 2.0],
                 [1.0, -2.0]])
cov_set = np.array([[[40., 0.0], [0.0, 40.]],
                  [[1.0, 0.5], [0.5, 0.5]],
                  [[0.5, 0.5], [0.5, 1.0]]])

# parameters for neural networks
neuron_1     = {0: K}
neuron_2_10  = {0: 10, 1: K}
neuron_2_100 = {0: 100, 1: K}
act_func_1 = {0: NN.softmax}
act_func_2 = {0: NN.relu, 1: NN.softmax}

In [ ]:
# sample object
# all sample will store in this sample object
G = Gaussian(N_set, mu_set, cov_set)

# initialize object visualization
visual = Visual(G.point, G.label, mu_set, cov_set)
visual.plot_sample().savefig("special/sample")

In [ ]:
string = "EM"

# train
method = EM(K)
method.train(G.train_point)
method.order_correction(G.valid_point, G.valid_label)

# result
accuracy = method.test(G.test_point, G.test_label)[0] * 100
time     = method.train_time
print("%02.04f  %02.4f" % (accuracy, time))

# decision boundary
visual.plot_DB(method).savefig("special/{}_DB".format(string))

In [ ]:
string = "Q(2-3)"

# train
method = NN(D, neuron_1, act_func_1, NN_type="QNN")
method.train(G.train_point, G.train_label, G.valid_point, G.valid_label)

# result
accuracy    = method.test(G.test_point, G.test_label)[0] * 100
time        = method.train_time

print("%02.04f  %02.4f" % (accuracy, time))

# decision boundary
visual.plot_DB(method).savefig("special/fig/{}_DB".format(string))

# train process
np.savetxt("special/iteration_{}.csv".format(string), method.iteration, delimiter=",")
np.savetxt("special/valid_loss_{}.csv".format(string), method.valid_loss, delimiter=",")

In [ ]:
string = "C(2-3)"

# train
method = NN(D, neuron_1, act_func_1, NN_type="LNN")
method.train(G.train_point, G.train_label, G.valid_point, G.valid_label)

# result
accuracy = method.test(G.test_point, G.test_label)[0] * 100
time     = method.train_time
print("%02.04f  %02.4f" % (accuracy, time))

# decision boundary
visual.plot_DB(method).savefig("special/{}_DB".format(string))

# train process
np.savetxt("special/iteration_{}.csv".format(string), method.iteration, delimiter=",")
np.savetxt("special/valid_loss_{}.csv".format(string), method.valid_loss, delimiter=",")

In [ ]:
string = "L10 "

max_accuracy = 100
max_time = 0
max_method = None

for i in range(100):
    # train
    method = NN(D, neuron_2_10, act_func_2, NN_type="LNN")
    method.train(G.train_point, G.train_label, G.valid_point, G.valid_label)

    # result
    accuracy    = method.test(G.test_point, G.test_label)[0] * 100
    time        = method.train_time

    if max_accuracy > accuracy:
        max_accuracy = accuracy
        max_time = time
        max_method = method
print("%02.04f  %02.4f" % (max_accuracy, max_time))

# decision boundary
visual.plot_DB(max_method).savefig("special/{}_DB".format(string))

# train process
np.savetxt("special/iteration_{}.csv".format(string), max_method.iteration, delimiter=",")
np.savetxt("special/valid_loss_{}.csv".format(string), max_method.valid_loss, delimiter=",")

In [ ]:
string = "L100"

max_accuracy = 100
max_time = 0
max_method = None

for i in range(100):
    # train
    method = NN(D, neuron_2_100, act_func_2, NN_type="LNN")
    method.train(G.train_point, G.train_label,
                 G.valid_point, G.valid_label, stop_point=200)

    # result
    accuracy    = method.test(G.test_point, G.test_label)[0] * 100
    time        = method.train_time

    if max_accuracy > accuracy:
        max_accuracy = accuracy
        max_time = time
        max_method = method

print("%02.04f  %02.4f" % (max_accuracy, max_time))

# decision boundary
visual.plot_DB(max_method).savefig("special/{}_DB".format(string))

# train process
np.savetxt("special/iteration_{}.csv".format(string), max_method.iteration, delimiter=",")
np.savetxt("special/valid_loss_{}.csv".format(string), max_method.valid_loss, delimiter=",")
